In [1]:
# %% [markdown]
# # Multi-Site Filter Data Summary
# This notebook extracts filter counts and data availability for all SPARTAN sites

# %%
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import the data loader
from data_loader_module import load_filter_database

# Configuration
PKL_PATH = '/Users/ahzs645/Github/aethmodular-clean/FTIR_HIPS_Chem/Filter Data/unified_filter_dataset.pkl'
SITES = ['ETAD', 'USPA', 'INDH', 'CHTS']


def normalize_filter_id(filter_id):
    """Normalize filter IDs so suffix variations like -# map to the same base filter."""
    if pd.isna(filter_id):
        return None
    filter_str = str(filter_id).strip()
    if not filter_str:
        return None
    parts = filter_str.split('-')
    if len(parts) >= 2:
        return f"{parts[0]}-{parts[1]}"
    return filter_str

print("📊 Multi-Site Filter Analysis")
print("="*60)

# %%
# Load database
loader = load_filter_database(PKL_PATH)
print(f"✅ Database loaded successfully")
print(f"📍 Sites available: {', '.join(loader.get_available_sites())}")

# %%
# Function to get detailed filter counts for a site
def get_site_filter_details(site_code, loader):
    """Get detailed filter information for a site"""
    
    # Get site summary first
    site_summary = loader.get_site_summary(site_code)
    
    if not site_summary:
        return None
    
    # Get unique filters by method
    filters_by_method = {}
    all_filters = set()
    
    # Check each data source
    for source in ['FTIR', 'HIPS', 'ChemSpec']:
        # Try to get any parameter from this source to count filters
        params = loader.list_available_parameters(site_code, source)
        if params:
            # Get data for first parameter to count unique filters
            data = loader.get_parameter_data(site_code, params[0])
            if len(data) > 0:
                # Check what columns are available
                if 'filter_id' in data.columns:
                    filter_col = 'filter_id'
                elif 'Filter_id' in data.columns:
                    filter_col = 'Filter_id'
                elif 'FilterID' in data.columns:
                    filter_col = 'FilterID'
                elif 'filter_ID' in data.columns:
                    filter_col = 'filter_ID'
                else:
                    # Try to find any column with 'filter' in the name
                    filter_cols = [col for col in data.columns if 'filter' in col.lower()]
                    if filter_cols:
                        filter_col = filter_cols[0]
                        print(f"   Using filter column: {filter_col} for {source}")
                    else:
                        print(f"   Warning: No filter ID column found for {source}")
                        print(f"   Available columns: {list(data.columns)[:5]}...")
                        continue
                
                raw_filters = data[filter_col].dropna().unique()
                normalized_filters = set()
                for filter_id in raw_filters:
                    normalized = normalize_filter_id(filter_id)
                    if normalized:
                        normalized_filters.add(normalized)

                if normalized_filters:
                    filters_by_method[source] = normalized_filters
                    all_filters.update(normalized_filters)
    
    # Find filters with complete measurements
    if filters_by_method:
        all_methods = list(filters_by_method.keys())
        if len(all_methods) > 1:
            complete_filters = filters_by_method[all_methods[0]]
            for method in all_methods[1:]:
                complete_filters = complete_filters.intersection(filters_by_method[method])
        else:
            complete_filters = filters_by_method[all_methods[0]] if all_methods else set()
    else:
        complete_filters = set()
    
    total_filters = len(all_filters) if all_filters else site_summary.get('unique_filters', 0)

    return {
        'total_filters': total_filters,
        'filters_by_method': {k: len(v) for k, v in filters_by_method.items()},
        'complete_filters': len(complete_filters),
        'date_range': (site_summary['date_range']['start'], site_summary['date_range']['end']),
        'total_measurements': site_summary['total_measurements']
    }

# %%
# Quick test to see what columns we actually have
print("🔍 Checking data structure...")
test_site = 'ETAD'
test_params = loader.list_available_parameters(test_site, 'FTIR')
if test_params:
    test_data = loader.get_parameter_data(test_site, test_params[0])
    print(f"Sample data columns from {test_site} FTIR:")
    print(f"   Columns: {list(test_data.columns)}")
    print(f"   Shape: {test_data.shape}")

# %%
# Analyze each site
site_summaries = {}

print("\n📋 DETAILED SITE ANALYSIS")
print("="*60)

for site in SITES:
    details = get_site_filter_details(site, loader)
    if details:
        site_summaries[site] = details
        
        print(f"\n🌍 {site} Site:")
        print(f"   Total unique filters: {details['total_filters']}")
        
        if details['filters_by_method']:
            print(f"   Filters by measurement type:")
            for method, count in details['filters_by_method'].items():
                print(f"      • {method}: {count} filters")
            
            print(f"   Filters with ALL measurements: {details['complete_filters']}")
            print(f"   Date range: {details['date_range'][0].date()} to {details['date_range'][1].date()}")
            print(f"   Total measurements: {details['total_measurements']}")
    else:
        print(f"\n🌍 {site} Site: No data available")

# %%
# Create summary table for presentation
print("\n📊 SUMMARY TABLE FOR PRESENTATION")
print("="*60)

# Build table data
table_data = []
for site in SITES:
    if site in site_summaries:
        s = site_summaries[site]
        
        # Get site location names
        site_names = {
            'ETAD': 'Addis Ababa, Ethiopia',
            'USPA': 'Pasadena/JPL, USA',
            'INDH': 'Delhi, India', 
            'CHTS': 'Beijing, China'
        }
        
        # Format date range
        date_str = f"{s['date_range'][0].strftime('%b %Y')} - {s['date_range'][1].strftime('%b %Y')}"
        
        # Format measurement counts
        meas_list = []
        for method in ['FTIR', 'HIPS', 'ChemSpec']:
            if method in s['filters_by_method']:
                count = s['filters_by_method'][method]
                if method == 'FTIR':
                    meas_list.append(f"FTIR-EC: {count}")
                elif method == 'HIPS':
                    meas_list.append(f"HIPS: {count}")
                elif method == 'ChemSpec':
                    meas_list.append(f"XRF/IC: {count}")
        meas_str = ", ".join(meas_list)
        
        table_data.append({
            'Site': site_names.get(site, site),
            'Filters': s['total_filters'],
            'Complete Sets': s['complete_filters'],
            'Date Range': date_str,
            'Measurements': meas_str
        })

# Create DataFrame for nice display
df = pd.DataFrame(table_data)
print("\n", df.to_string(index=False))

print("\n*Complete Sets = filters measured by all available methods at that site")

# %%
# Generate data for PowerPoint slide
print("\n📊 DATA FOR POWERPOINT SLIDE")
print("="*60)
print("\nCopy this table for your presentation:\n")

for row in table_data:
    print(f"{row['Site']}:")
    print(f"  • Total Filters: {row['Filters']}")
    print(f"  • Complete Sets: {row['Complete Sets']}")
    print(f"  • Date Range: {row['Date Range']}")
    print(f"  • Available: {row['Measurements']}")
    print()

# %%
# Analysis for transferability
print("\n🔄 TRANSFERABILITY ASSESSMENT")
print("="*60)

# Check for overlapping time periods
print("\n📅 Overlapping Time Periods:")
all_dates = []
for site in SITES:
    if site in site_summaries:
        start, end = site_summaries[site]['date_range']
        all_dates.append((site, start, end))

if len(all_dates) > 1:
    # Find overlap periods
    for i, (site1, start1, end1) in enumerate(all_dates):
        for site2, start2, end2 in all_dates[i+1:]:
            overlap_start = max(start1, start2)
            overlap_end = min(end1, end2)
            
            if overlap_start < overlap_end:
                overlap_days = (overlap_end - overlap_start).days
                print(f"   {site1} ↔ {site2}: {overlap_start.date()} to {overlap_end.date()} ({overlap_days} days)")

# %%
# Filter availability comparison
print("\n📊 Measurement Type Coverage:")
all_methods = ['FTIR', 'HIPS', 'ChemSpec']

for method in all_methods:
    print(f"\n{method}:")
    for site in SITES:
        if site in site_summaries:
            count = site_summaries[site]['filters_by_method'].get(method, 0)
            total = site_summaries[site]['total_filters']
            pct = (count / total * 100) if total > 0 else 0
            print(f"   {site}: {count}/{total} filters ({pct:.1f}% coverage)")

# %%
# Summary statistics for presentation
print("\n📈 SUMMARY STATISTICS")
print("="*60)

total_filters_all_sites = sum(s['total_filters'] for s in site_summaries.values())
total_complete_sets = sum(s['complete_filters'] for s in site_summaries.values())

print(f"Total filters across all sites: {total_filters_all_sites}")
print(f"Total complete measurement sets: {total_complete_sets}")
print(f"Overall completeness: {total_complete_sets/total_filters_all_sites*100:.1f}%")

# %%
print("\n✅ Analysis complete!")
print("\n📝 Key Findings:")
print("   • Each filter can be measured by up to 3 different methods (FTIR, HIPS, XRF/IC)")
print("   • Not all filters have complete measurements across all methods")
print("   • ETAD has most complete recent dataset (Dec 2022 - Sep 2024)")
print("   • Overlapping time periods enable cross-site validation")

📊 Multi-Site Filter Analysis
Loading complete filter dataset from /Users/ahzs645/Github/aethmodular-clean/FTIR_HIPS_Chem/Filter Data/unified_filter_dataset.pkl...
Dataset loaded successfully!
   Total measurements: 44,493
   Unique filters: 1,603
   Sites: CHTS, ETAD, INDH, USPA
   Date range: 2013-06-28 to 2024-12-08
   Data sources: ChemSpec, FTIR, HIPS
✅ Database loaded successfully
📍 Sites available: CHTS, ETAD, INDH, USPA
🔍 Checking data structure...
Sample data columns from ETAD FTIR:
   Columns: ['SampleDate', 'Concentration', 'Concentration_Units', 'FilterId', 'MDL', 'Uncertainty']
   Shape: (190, 6)

📋 DETAILED SITE ANALYSIS
   Using filter column: FilterId for FTIR
   Using filter column: FilterId for HIPS
   Using filter column: FilterId for ChemSpec

🌍 ETAD Site:
   Total unique filters: 190
   Filters by measurement type:
      • FTIR: 190 filters
      • HIPS: 190 filters
      • ChemSpec: 188 filters
   Filters with ALL measurements: 188
   Date range: 2022-12-07 to 2024